# Natural Language Inference

In this lab, we are going to simply experiment with a natural language inference (NLI) model.

## Importing the dependencies

First, we are going to import all the dependencies that we will need for this lab. If you cannot run the following code cell, do not forget to [create an environment](https://www.freecodecamp.org/news/how-to-setup-virtual-environments-in-python/), to install the dependencies inside of it (using the command `pip install -r requirements.txt`) and to use it as your Jupyter kernel.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['HF_HOME'] = os.getcwd() + "/cache/"

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Identifying the best device to run the model

Since we are going to perform a computing-intensive task, we must identify the most efficient device available to perform it. We do so using PyTorch, which is the back-end that we will use in this lab. We prioritize NVIDIA GPUs with CUDA installed, then Apple Silicon GPUs, and finally CPUs if none of the above is found.

If you need help installing the relevant version of PyTorch: https://pytorch.org/get-started/locally/

If you have a NVIDIA GPU but you don't know whether you have CUDA installed or not, type the following command:

```bash
nvcc --version
```

If you have it installed, you should see the CUDA version installed on your computer. Otherwise, you should install a PyTorch-compatible version (as listed [here](https://pytorch.org/get-started/locally/), row "Stable CUDA").

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')

print(device)

Now let's download and prepare the language model that we will use! This model is a fine-tuning of RoBERTa<sub>LARGE</sub> on NLI datasets.

In [ ]:
model_nli_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
tokenizer_nli = AutoTokenizer.from_pretrained(model_nli_name)
model_nli = AutoModelForSequenceClassification.from_pretrained(model_nli_name).to(device)

## Defining the texts for our experiment

In [ ]:
premise = "My cat is in the tree"
hypothesis = "There is at least one cat in the tree"

In [ ]:
encoded_text = tokenizer_nli(premise, hypothesis, return_tensors='pt', max_length=512, truncation=True).to(device)
logits = model_nli(encoded_text["input_ids"])["logits"][0]
probs = torch.softmax(logits, -1).tolist()
dict_predicted_probability = {'entailment': probs[0], 'neutral': probs[1], 'contradiction': probs[2]}

dict_predicted_probability

That's it, we can now perform NLI on texts! Let's do a few tests!